# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [1]:
# Use this cell to write your code for Task 1
import pandas as pd
pd.set_option('display.max_columns', None) 
clean_data = pd.read_csv('loyalty.csv')
print(clean_data.head())
print(clean_data.info())
print(clean_data.isna().sum())
#examining categorical columns
print(clean_data['region'].value_counts())
print(clean_data['loyalty_years'].value_counts())
print(clean_data['joining_month'].value_counts())
print(clean_data['promotion'].value_counts())

   customer_id   spend first_month  items_in_first_month              region  \
0            1  132.68        15.3                     5        Asia/Pacific   
1            2  106.45        16.2                    14        Asia/Pacific   
2            3  123.16       25.26                     7  Middle East/Africa   
3            4  130.60       24.74                     8  Middle East/Africa   
4            5  130.41       25.59                     8  Middle East/Africa   

  loyalty_years joining_month promotion  
0          5-10           Nov        No  
1           0-1           Feb       Yes  
2           10+           Dec       Yes  
3           3-5           Apr        No  
4           3-5           Apr       Yes  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customer_id           1246 non-null   int64  
 1   spe

In [2]:
#first month has the wrong data type
clean_data['first_month'] = pd.to_numeric(clean_data['first_month'], errors='coerce')

#joining month has missing values. replacing the missing values with unknown
clean_data['joining_month'] = clean_data['joining_month'].fillna('Unknown')

#first_month has missing values
clean_data['first_month'] = clean_data['first_month'].fillna(0)

#promotion has capitalized values
clean_data['promotion'] = clean_data['promotion'].str.lower().str.capitalize()

#final check
print(clean_data.head())
print(clean_data.info())
print(clean_data.isna().sum())

   customer_id   spend  first_month  items_in_first_month              region  \
0            1  132.68        15.30                     5        Asia/Pacific   
1            2  106.45        16.20                    14        Asia/Pacific   
2            3  123.16        25.26                     7  Middle East/Africa   
3            4  130.60        24.74                     8  Middle East/Africa   
4            5  130.41        25.59                     8  Middle East/Africa   

  loyalty_years joining_month promotion  
0          5-10           Nov        No  
1           0-1           Feb       Yes  
2           10+           Dec       Yes  
3           3-5           Apr        No  
4           3-5           Apr       Yes  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customer_id           1246 non-null   int64  
 1

# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [3]:
# Use this cell to write your code for Task 2
#creating by_years
by_years = clean_data[['loyalty_years','spend']]
#grouping by_years df by spend and calculating the average to create the avg_spend column
spend_by_years = by_years.groupby('loyalty_years')['spend'].agg(['mean','var']).reset_index().rename(columns={'mean':'avg_spend','var':'var_spend'})
#rounding the avg_spend column to 2 decimal places
spend_by_years['avg_spend'] = round(spend_by_years['avg_spend'],2)
#rounding the var_spend column to 2 decimal places
spend_by_years['var_spend'] = round(spend_by_years['var_spend'],2)
#eaxmining the changes
print(spend_by_years)

  loyalty_years  avg_spend  var_spend
0           0-1     110.56       9.30
1           1-3     129.31       9.65
2           10+     117.41      16.72
3           3-5     124.55      11.09
4          5-10     135.15      14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [4]:
# Use this cell to write your code for Task 3
train_data = pd.read_csv('train.csv')
#Get feature and target variables
X = train_data.drop('spend', axis=1)
y = train_data['spend']
#Convering categorical variables to numerical using one-hot encoding
categorical_cols = X.select_dtypes(include=['object'])
#Applying the one-hot encoding
categorical_cols = pd.get_dummies(categorical_cols, drop_first=True)
#Replacing all boolean columns with 1s and 0s
categorical_cols.replace({'Yes': 1, 'No': 0}, inplace=True)
#selecting the columns with numerical data
numerical_cols = X.select_dtypes(include=['int64','float64'])
#concatenating the numerical and categorical columns to form one dataframe
X = pd.concat([numerical_cols, categorical_cols], axis=1)
#instantiating the model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#fitting the model to target 
model.fit(X,y)
#preprocessing test data the same as train data
test_data = pd.read_csv('test.csv')
print(test_data.head())
# Save the customer_id for the final result
customer_ids = test_data['customer_id']
#separating categorical and numerical columns
test_categorical = test_data.select_dtypes(include=['object'])
test_numerical = test_data.select_dtypes(include=['int64','float64'])
#applying one-hot encoding
test_categorical_encoded = pd.get_dummies(test_categorical, drop_first=True)
test_categorical_encoded.replace({'Yes':1,'No':0}, inplace=True)
#concatenating numerical and categorical data
X_test = pd.concat([test_numerical,test_categorical_encoded], axis=1)
#aligning columns to ensure columns are the same in test and train data
X_test = X_test.reindex(columns=X.columns, fill_value=0)
#making predictions
predictions = model.predict(X_test)
#creating the base_result dataframe
base_result = pd.DataFrame({
    'customer_id':customer_ids,
    'spend':predictions
})
#examining the result
print(base_result.head())


   customer_id  first_month  items_in_first_month              region  \
0            5        20.43                     8              Europe   
1            7        20.90                     7            Americas   
2           16        19.99                     7        Asia/Pacific   
3           17        23.55                    13  Middle East/Africa   
4           19        25.85                    11              Europe   

  loyalty_years joining_month promotion  
0          5-10           Jul       Yes  
1           3-5           Jul        No  
2          5-10           Jul       Yes  
3           3-5           Aug        No  
4           3-5           Apr       Yes  
   customer_id       spend
0            5  140.711949
1            7  148.742502
2           16  140.822343
3           17  150.659149
4           19  153.641091


# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [ ]:
# Use this cell to write your code for Task 4
train_data = pd.read_csv('train.csv')
#Get feature and target variables
X = train_data.drop('spend', axis=1)
y = train_data['spend']
#Convering categorical variables to numerical using one-hot encoding
categorical_cols = X.select_dtypes(include=['object'])
#Applying the one-hot encoding
categorical_cols = pd.get_dummies(categorical_cols, drop_first=True)
#Replacing all boolean columns with 1s and 0s
categorical_cols.replace({'Yes': 1, 'No': 0}, inplace=True)
#selecting the columns with numerical data
numerical_cols = X.select_dtypes(include=['int64','float64'])
#concatenating the numerical and categorical columns to form one dataframe
X = pd.concat([numerical_cols, categorical_cols], axis=1)
#instantiating the model 
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
#fitting the model to target 
model.fit(X,y)
#preprocessing test data the same as train data
test_data = pd.read_csv('test.csv')
# Save the customer_id for the final result
customer_ids = test_data['customer_id']
#separating categorical and numerical columns
test_categorical = test_data.select_dtypes(include=['object'])
test_numerical = test_data.select_dtypes(include=['int64','float64'])
#applying one-hot encoding
test_categorical_encoded = pd.get_dummies(test_categorical, drop_first=True)
test_categorical_encoded.replace({'Yes':1,'No':0}, inplace=True)
#concatenating numerical and categorical data
X_test = pd.concat([test_numerical,test_categorical_encoded], axis=1)
#aligning columns to ensure columns are the same in test and train data
X_test = X_test.reindex(columns=X.columns, fill_value=0)
#making predictions
predictions = model.predict(X_test)
#creating the base_result dataframe
compare_result = pd.DataFrame({
    'customer_id':customer_ids,
    'spend':predictions
})
#examining the result
print(compare_result.head())

   customer_id   spend
0            5  140.59
1            7  148.41
2           16  141.02
3           17  151.16
4           19  153.68
